In [1]:
require 'open-uri'
require 'json'
require 'daru'
require 'gsl'
require 'distribution'
require 'sqlite3' ##This should return true if you have SQLite3 installed
require './regression'
include Regression

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

Object

In [2]:
dir = "#{ENV['HOME']}/cs6140/final_project"
db = SQLite3::Database.new "#{dir}/credit_risk_data.db"
db.results_as_hash = true

true

In [3]:
begin db.execute("drop table cross_validation"); rescue; end
sql = <<SQL
create temp table cross_validation as 
  select sk_id_curr, abs(random()) % 10 as fold
  from application_train
  order by random()
SQL
db.execute(sql)

[]

In [4]:
sql = <<SQL
create temp view example_regression as 
  select a.sk_id_curr, 
  target,
  fold, ext_source_1, ext_source_2, ext_source_3
  from application_train a inner join cross_validation c on a.sk_id_curr = c.sk_id_curr
SQL
begin db.execute("drop view example_regression"); rescue; end
db.execute(sql)

[]

In [5]:
column_types = Hash.new
db.execute("PRAGMA table_info(application_train);").each do |row|
  column_types[row["name"]] = row["type"]
end
1

1

In [6]:
def plot x, y
  Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "X"
    plot.y_label "Y"
  end
end

:plot

In [7]:
class NaiveBayesModel
  def func dataset, w
    nll = 0.0
    
    dataset.each do |i|
      nll -= Math.log(w["#{i["label"]}"])
      i["features"].each do |j,k|
        next if k.nil? || k.eql?("") 
        nll += 0.5*(Math.log(2*(Math::PI)*(w["#{i["label"]}_sigma_#{j}"]**2)) + ((k-(w["#{i["label"]}_mu_#{j}"])**2)/(2*(w["#{i["label"]}_sigma_#{j}"]**2))))
      end
    end
    
    return nll
  end
  
  @@classes = []
  
  def grad dataset, w
    g = Hash.new {|h,k| h[k] = 0.0}
    
    if @@classes.length ==0
      dataset.each do |i|
        @@classes << i["label"]
      end
      @@classes.uniq!
    end
    
    dataset.each do |i|
      g["#{i["label"]}"] -= 1.0/w["#{i["label"]}"]
      i["features"].each do |j,k|
        sigma = w["#{i["label"]}_sigma_#{j}"]
        mu = w["#{i["label"]}_mu_#{j}"]
        next if k.nil? || k.eql?("")
        g["#{i["label"]}_mu_#{j}"] -= (k-mu)/(sigma**2)
        g["#{i["label"]}_sigma_#{j}"] += (0.5/sigma**2) - ((k-mu)/(2*(sigma**4)))
      end
    end
    
    return g
  end
  
  def predict row, w
    score = Hash.new{|h,k| h[k]=0.0}
    
    best_score = 0.0
#     best_lbl = ""
    
    @@classes.each do |i|
      score[i] += Math.log(w[i])
      row["features"].each do |j,k|
        next if k.nil? || k.eql?("")
        score[i] -= 0.5*(Math.log(2*(Math::PI)*(w["#{i}_sigma_#{j}"]**2)) + ((k-(w["#{i}_mu_#{j}"])**2)/(2*(w["#{i}_sigma_#{j}"]**2))))
      end
    end
    best_score = score.values.max_by{ |x| x**2}
    best_lbl = score.key(best_score)
    return {best_lbl => best_score}
  end

  def adjust w
    w.each_key do |fname|
      next if fname.class != Numeric
      if fname.inclue?("mu") || fname.include?("sigma")
      else
        w[fname] = [[0.001, w[fname]].max, 0.999].min
      end
    end
  end
end

:adjust

In [8]:
def gradient_descent x, w, obj, learning_rate = 1e-4, rmse_tol = 1e-3, max_iters = 1000
  iters = []
  rmses = []
  zscore_bias_data = x
  w = obj.adjust w
  rms = obj.func(zscore_bias_data, w)
  rmses << rms
  itr = 1
  iters << itr
  rmse_diff = 100
  while itr < max_iters && rmse_diff > rmse_tol do
    dw = obj.grad(zscore_bias_data, w)
    w = update_weights(w, dw, learning_rate)
    w = obj.adjust w
    rms = obj.func(zscore_bias_data, w)
    rmse_diff = rmses.last - rms
    rmses << rms
    
    itr+=1
    iters << itr
  end
  
  return [iters, rmses, w, zscore_bias_data]
end

:gradient_descent

In [9]:
def find_nb_score model, w, data
  scores = []
  data.each do |row|
    corrected_row = row
    prediction = model.predict(corrected_row, w)
    scores << [prediction.collect{|j,k| k}[0],corrected_row["label"]]
  end
  return scores
end
1

1

In [10]:
def roc_curve(scores)
  totals = Hash.new {|h,k| h[k] = 0.0} 
  scores.each do |j,k|
    totals[k] += 1.0
  end
  
  computed = Hash.new{|h,k| h[k]=0.0}
  false_positive = [0.0]
  true_positive = [0.0]
  auc = 0.0
  
  scores.sort_by{ |scor,pred| -scor}.each do |score,predicted|
    computed[predicted] += 1
    fpr = computed[0]/totals[0]
    tpr = computed[1]/totals[1]
    auc += 0.5*(fpr-false_positive.last)*(tpr+true_positive.last)
    false_positive << fpr
    true_positive << tpr
  end
  return[false_positive,true_positive,auc]
end

:roc_curve

In [11]:
batch_data = db.execute("SELECT SK_ID_CURR, TARGET, CNT_CHILDREN, FLAG_MOBIL, FLAG_EMP_PHONE, FLAG_WORK_PHONE, FLAG_CONT_MOBILE, FLAG_PHONE, FLAG_EMAIL, CNT_FAM_MEMBERS, REGION_RATING_CLIENT, REGION_RATING_CLIENT_W_CITY, HOUR_APPR_PROCESS_START, REG_REGION_NOT_LIVE_REGION, REG_REGION_NOT_WORK_REGION, LIVE_REGION_NOT_WORK_REGION, REG_CITY_NOT_LIVE_CITY, REG_CITY_NOT_WORK_CITY, LIVE_CITY_NOT_WORK_CITY, NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_TYPE_SUITE, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE, WEEKDAY_APPR_PROCESS_START, ORGANIZATION_TYPE, EMERGENCYSTATE_MODE, OBS_30_CNT_SOCIAL_CIRCLE, DEF_30_CNT_SOCIAL_CIRCLE, OBS_60_CNT_SOCIAL_CIRCLE, DEF_60_CNT_SOCIAL_CIRCLE FROM application_train limit 50000;")
data = labelData batch_data , column_types
1

1

In [12]:
nb = NaiveBayesModel.new
w = Hash.new {|h,k| h[k] = 0.5}
iters, lik, norms, w, zb = gradient_descent data, w, nb, 1e-3, 1.0, 100
plot(iters, lik)

#<Nyaplot::Plot:0x007fa005f1b330 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fa005f18798 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"688e188d-6bca-4c59-8d92-1920e24d29ba"}, @xrange=[1, 3], @yrange=[11371465.402160412, 6453293924.705217]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[1, 3], :yrange=>[11371465.402160412, 6453293924.705217]}}>

### Naive Bayes
*Graph between Iterations and RMSE*
![naive](Naive_rmse.png)



In [13]:
sql = "SELECT SK_ID_CURR, TARGET, CNT_CHILDREN, FLAG_MOBIL, FLAG_EMP_PHONE, FLAG_WORK_PHONE, FLAG_CONT_MOBILE, FLAG_PHONE, FLAG_EMAIL, CNT_FAM_MEMBERS, REGION_RATING_CLIENT, REGION_RATING_CLIENT_W_CITY, HOUR_APPR_PROCESS_START, REG_REGION_NOT_LIVE_REGION, REG_REGION_NOT_WORK_REGION, LIVE_REGION_NOT_WORK_REGION, REG_CITY_NOT_LIVE_CITY, REG_CITY_NOT_WORK_CITY, LIVE_CITY_NOT_WORK_CITY, NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_TYPE_SUITE, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE, WEEKDAY_APPR_PROCESS_START, ORGANIZATION_TYPE, EMERGENCYSTATE_MODE, OBS_30_CNT_SOCIAL_CIRCLE, DEF_30_CNT_SOCIAL_CIRCLE, OBS_60_CNT_SOCIAL_CIRCLE, DEF_60_CNT_SOCIAL_CIRCLE FROM application_train limit 10000;"
scores = find_nb_score NaiveBayesModel.new, w, data
f_p, t_p, auc = roc_curve scores
puts auc
plot f_p, t_p

0.490662350008394


### Naive Bayes
*Accuracy is 0.490662350008394*
![naive](Naive_roc.png)
